### Courtesy and acknowledgement

https://platform.openai.com/docs/guides/function-calling

https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models

### Set up Sql database

In [1]:
# Import the sqlite3 library
import sqlite3

conn = sqlite3.connect('order_information.db',check_same_thread=False)
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS order_data')

cursor.execute('''
CREATE TABLE order_data (
    order_no INTEGER PRIMARY KEY,
    customer_name TEXT
    )
''')

# Insert rows of data into the table
rows_to_insert = [
    (1, 'Alice'),
    (2, 'Bob'),
    (3, 'Charlie')
]

# Use executemany() to insert multiple rows
cursor.executemany('INSERT INTO order_data VALUES (?, ?)', rows_to_insert)

# Commit the changes
conn.commit()


In [2]:

cursor.execute('DROP TABLE IF EXISTS item_data')


cursor.execute('''
CREATE  TABLE item_data (
    order_no INTEGER ,
    item_no INTEGER,
    item_name TEXT,
    item_amount float,
    status TEXT
    )
''')

# Insert rows of data into the table
rows_to_insert = [
    (1, 100,'TV',600, 'Shipped'),
    (1, 101,'Fridge',200, 'In Process'),
    (2, 200,'VR glasses',400, 'Shipped'),
    (2, 201,'Pet robot',100, 'Shipped'),
    (3, 100,'TV',100, 'Cancelled')
]

# Use executemany() to insert multiple rows
cursor.executemany('INSERT INTO item_data VALUES (?, ?, ?, ?, ?)', rows_to_insert)

# Commit the changes
conn.commit()

# Always close the connection when done
#conn.close()

In [3]:
cursor.execute('DROP TABLE IF EXISTS product_data')

cursor.execute('''
CREATE TABLE product_data (
    product_no INTEGER PRIMARY KEY,
    product_name TEXT
    )
''')

# Insert rows of data into the table
rows_to_insert = [
    (1, 'TV'),
    (2, 'Fridge'),
    (3, 'VR glasses'),
    (4, 'Pet robot'),
    (5, 'Fitness watch')
]

# Use executemany() to insert multiple rows
cursor.executemany('INSERT INTO product_data VALUES (?, ?)', rows_to_insert)

# Commit the changes
conn.commit()

### Set up Sql metadata fetch functions

In [4]:
def find_tables(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names


def find_columns(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names


def find_database(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in find_tables(conn):
        columns_names = find_columns(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts

In [5]:
database_schema_dict = find_database(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

In [20]:
print (database_schema_string)

Table: order_data
Columns: order_no, customer_name
Table: item_data
Columns: order_no, item_no, item_name, item_amount, status
Table: product_data
Columns: product_no, product_name


### Set up tool to specify the function 

In [6]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "invoke_database_api_call",
            "description": "Use this function to answer user questions about data. Input should be a fully formed SQL query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": f"""
                                SQL query extracting info to answer the user's question.
                                SQL should be written using this database schema:
                                {database_schema_string}
                                The query should be returned in plain text, not in JSON.
                                """,
                    }
                },
                "required": ["query"],
            },
        }
    }
]

### Set up functions to invoke_database_api_call

In [7]:
def invoke_database_api_call(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

def execute_function(message):
    if message.tool_calls[0].function.name == "invoke_database_api_call":
        query = json.loads(message.tool_calls[0].function.arguments)["query"]
        results = invoke_database_api_call(conn, query)
    else:
        results = f"Error: function {message.tool_calls[0].function.name} does not exist"
    return results

In [22]:
from openai import OpenAI
import json
import getpass
import os


os.environ["OPENAI_API_KEY"] = getpass.getpass()

client = OpenAI()

 ········


In [9]:
from termcolor import colored  


model = "gpt-3.5-turbo-0613"

def llm_response_request(messages, tools=None, tool_choice=None, model=model):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [10]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "blue",
        "user": "grey",
        "assistant": "cyan",
        "function": "red",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))

### Use case 1 : Ask about order status

In [33]:
messages = []
messages.append({"role": "system", "content": "Answer user questions by generating SQL queries against the order_information Database."})
messages.append({"role": "user", "content": "Hi, My name is Bob, Can you let me know the status of my order items?"})
chat_response = llm_response_request(messages, tools)
assistant_message = chat_response.choices[0].message
print ('assistant_message',assistant_message)
print ('\nassistant_message.tool_calls[0].function',assistant_message.tool_calls[0].function)
print ('\nstr(assistant_message.tool_calls[0].function',str(assistant_message.tool_calls[0].function))
print ('\njson loads assistant_message',json.loads(str(assistant_message.tool_calls[0].function.arguments)))
assistant_message.content = str(assistant_message.tool_calls[0].function)
#print ('\nassistant_message',assistant_message)
messages.append({"role": assistant_message.role, "content": assistant_message.content})
if assistant_message.tool_calls:
    results = execute_function(assistant_message)
    messages.append({"role": "function", "tool_call_id": assistant_message.tool_calls[0].id, "name": assistant_message.tool_calls[0].function.name, "content": results})
print ('\nmessages\n')
pretty_print_conversation(messages)

assistant_message ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_txhpjlqIIU7kZax5gdJyLckk', function=Function(arguments='{\n"query": "SELECT item_name, status FROM item_data WHERE order_no = (SELECT order_no FROM order_data WHERE customer_name = \'Bob\')"\n}', name='invoke_database_api_call'), type='function')])

assistant_message.tool_calls[0].function Function(arguments='{\n"query": "SELECT item_name, status FROM item_data WHERE order_no = (SELECT order_no FROM order_data WHERE customer_name = \'Bob\')"\n}', name='invoke_database_api_call')

str(assistant_message.tool_calls[0].function Function(arguments='{\n"query": "SELECT item_name, status FROM item_data WHERE order_no = (SELECT order_no FROM order_data WHERE customer_name = \'Bob\')"\n}', name='invoke_database_api_call')

json loads assistant_message {'query': "SELECT item_name, status FROM item_data WHERE order_no = (SELECT order_no FROM order_data WHE

In [12]:
messages

[{'role': 'system',
  'content': 'Answer user questions by generating SQL queries against the order_information Database.'},
 {'role': 'user',
  'content': 'Hi, My name is Bob, Can you let me know the status of my order items?'},
 {'role': 'assistant',
  'content': 'Function(arguments=\'{\\n"query": "SELECT item_no, item_name, status FROM item_data WHERE order_no IN(SELECT order_no FROM order_data WHERE customer_name = \\\'Bob\\\')"\\n}\', name=\'invoke_database_api_call\')'},
 {'role': 'function',
  'tool_call_id': 'call_7e0v0u29hScwTveWd04UVJKK',
  'name': 'invoke_database_api_call',
  'content': "[(200, 'VR glasses', 'Shipped'), (201, 'Pet robot', 'Shipped')]"}]

In [13]:
final_message = []
final_message.append({"role": "system", "content": "summarize in text format only the final status information provided by the content in ask_database function results provided in the messages."})

c = f"{messages}"
c 
final_message.append({"role": "user", "content": f"{c}"})
final_message


[{'role': 'system',
  'content': 'summarize in text format only the final status information provided by the content in ask_database function results provided in the messages.'},
 {'role': 'user',
  'content': '[{\'role\': \'system\', \'content\': \'Answer user questions by generating SQL queries against the order_information Database.\'}, {\'role\': \'user\', \'content\': \'Hi, My name is Bob, Can you let me know the status of my order items?\'}, {\'role\': \'assistant\', \'content\': \'Function(arguments=\\\'{\\\\n"query": "SELECT item_no, item_name, status FROM item_data WHERE order_no IN(SELECT order_no FROM order_data WHERE customer_name = \\\\\\\'Bob\\\\\\\')"\\\\n}\\\', name=\\\'invoke_database_api_call\\\')\'}, {\'role\': \'function\', \'tool_call_id\': \'call_7e0v0u29hScwTveWd04UVJKK\', \'name\': \'invoke_database_api_call\', \'content\': "[(200, \'VR glasses\', \'Shipped\'), (201, \'Pet robot\', \'Shipped\')]"}]'}]

In [14]:
response = client.chat.completions.create(
            model=model,
            messages=final_message
        )

In [15]:
        
print ('final response\n')
print (colored(response.choices[0].message.content,'green'))

final response

The final status information provided by the content in the ask_database function results is:
- Order item with item number 200, item name 'VR glasses', and status 'Shipped'
- Order item with item number 201, item name 'Pet robot', and status 'Shipped'


### Use case 2 : Ask about product catalog

In [16]:
messages = []
messages.append({"role": "system", "content": "Answer user questions by generating SQL queries against the order_information Database."})

messages.append({"role": "user", "content": "Hi, Can you let me know what are all the product you sell?"})
chat_response = llm_response_request(messages, tools)
assistant_message = chat_response.choices[0].message
assistant_message.content = str(assistant_message.tool_calls[0].function)
messages.append({"role": assistant_message.role, "content": assistant_message.content})
if assistant_message.tool_calls:
    results = execute_function(assistant_message)
    messages.append({"role": "function", "tool_call_id": assistant_message.tool_calls[0].id, "name": assistant_message.tool_calls[0].function.name, "content": results})
pretty_print_conversation(messages)

system: Answer user questions by generating SQL queries against the order_information Database.

user: Hi, Can you let me know what are all the product you sell?

assistant: Function(arguments='{\n  "query": "SELECT product_name FROM product_data"\n}', name='invoke_database_api_call')

function (invoke_database_api_call): [('TV',), ('Fridge',), ('VR glasses',), ('Pet robot',), ('Fitness watch',)]



In [18]:
final_message = []
final_message.append({"role": "system", "content": "summarize in text format only the final status information provided by the content in ask_database function results provided in the messages. list items in bulleted format"})

c = f"{messages}"
c 
final_message.append({"role": "user", "content": f"{c}"})
final_message
response = client.chat.completions.create(
            model=model,
            messages=final_message
        )
print ('final response\n')
print (colored(response.choices[0].message.content,'green'))

final response

- The user asked: "Can you let me know what are all the products you sell?"
- The assistant responded by calling the function `invoke_database_api_call` with the argument `query = "SELECT product_name FROM product_data"`.
- The function `invoke_database_api_call` returned a list of products: 
  - TV
  - Fridge
  - VR glasses
  - Pet robot
  - Fitness watch
